In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import pandas as pd
import re
import spacy
from tqdm.notebook import tqdm
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline


## Load the Dataset

In [ ]:
dataset = load_dataset("CarperAI/openai_summarize_tldr")
train_dataset = dataset['train']
test_dataset = dataset['test']
train_dataset[1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'prompt': 'SUBREDDIT: r/loseit\nTITLE: SV & NSV! Keeping on keeping on.\nPOST: 30F, 5\'6". SW: 236 GW: 150 CW: 219\n\nI weigh myself weekly and measure myself monthly. I\'d hit a plateau the last four weeks or so where I was stuck at 222. Felt like kind of a bummer, but knew it\'s because I haven\'t been as strict as I should with my diet, and the last week and a half have been crazy with life things, so I haven\'t been exercising as frequently as I\'ve gotten used to. When I weighed myself as normal on Monday, I was kind of disappointed to see the scale not budging and figured it was time to buckle down again and really watch my diet. Today was my measure-in day, and I\'ve felt cruddy in general since Monday because I caught some chest congestion/cold bug over the weekend. I get on the scale...it says 219. Whaaaaat? I take my measurements, which are down slightly from last month, and with an total-body loss of 8 inches from my starting point on 12/23/14! Some of my clothes have been 

In [ ]:
test_dataset.column_names

['prompt', 'label']

## Inspect Data

In [ ]:
# inspect column names
print(f"Dataset Column Names:\n {dataset.column_names}")

# create a DataFrame and inspect
pd.set_option('display.max_colwidth', None) # For better insight
df = pd.DataFrame(train_dataset)
df.iloc[1]


Dataset Column Names:
 {'train': ['prompt', 'label'], 'test': ['prompt', 'label'], 'valid': ['prompt', 'label']}


,1
prompt,"SUBREDDIT: r/loseit\nTITLE: SV & NSV! Keeping on keeping on.\nPOST: 30F, 5'6"". SW: 236 GW: 150 CW: 219\n\nI weigh myself weekly and measure myself monthly. I'd hit a plateau the last four weeks or so where I was stuck at 222. Felt like kind of a bummer, but knew it's because I haven't been as strict as I should with my diet, and the last week and a half have been crazy with life things, so I haven't been exercising as frequently as I've gotten used to. When I weighed myself as normal on Monday, I was kind of disappointed to see the scale not budging and figured it was time to buckle down again and really watch my diet. Today was my measure-in day, and I've felt cruddy in general since Monday because I caught some chest congestion/cold bug over the weekend. I get on the scale...it says 219. Whaaaaat? I take my measurements, which are down slightly from last month, and with an total-body loss of 8 inches from my starting point on 12/23/14! Some of my clothes have been feeling a bit looser as of late and now I know it's just not in my head. I'm now the lightest and smallest I've been since right around high school!\nTL;DR:"
label,"Progress is still happening, even when you think it might not be! Don't get discouraged, even if your journey seems to be going slowly. Don't give up, warriors."


## Preprocess the Data

In [ ]:
# we only want to use the prompt column
df = df.iloc[:, [0]]

df.iloc[:2]

,prompt
0,SUBREDDIT: r/relationships\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mother has been out of my life since then so there's been alot of progress. Being stronger after learning some lessons there been more insight about that time of my life but when I see him or a picture everything comes back. The emotions and memories bring me back down. \n\nHis friends (both girls) are on my facebook because we get along well which is hard to find and I know they'll always have his back. But seeing him in a picture or talking to him at a convention having a conversation is tough. Crying confront of my current boyfriend is something I want to avoid. \n\nSo I've been thinking that I have to cut contact with these girls because it's time to move on because it's healthier. It's best to avoid him as well. But will they be insulted? Will they accept it? Is there going to be awkwardness? I'm not sure if it's the right to do and could use some outside opinions.\nTL;DR:
1,"SUBREDDIT: r/loseit\nTITLE: SV & NSV! Keeping on keeping on.\nPOST: 30F, 5'6"". SW: 236 GW: 150 CW: 219\n\nI weigh myself weekly and measure myself monthly. I'd hit a plateau the last four weeks or so where I was stuck at 222. Felt like kind of a bummer, but knew it's because I haven't been as strict as I should with my diet, and the last week and a half have been crazy with life things, so I haven't been exercising as frequently as I've gotten used to. When I weighed myself as normal on Monday, I was kind of disappointed to see the scale not budging and figured it was time to buckle down again and really watch my diet. Today was my measure-in day, and I've felt cruddy in general since Monday because I caught some chest congestion/cold bug over the weekend. I get on the scale...it says 219. Whaaaaat? I take my measurements, which are down slightly from last month, and with an total-body loss of 8 inches from my starting point on 12/23/14! Some of my clothes have been feeling a bit looser as of late and now I know it's just not in my head. I'm now the lightest and smallest I've been since right around high school!\nTL;DR:"


In [ ]:
total_rows = df.shape[0]
print("Total rows in the DataFrame:", total_rows)

Total rows in the DataFrame: 116722


In [ ]:
# Filtering out the subreddit categories
df['subreddit_category'] = df['prompt'].str.extract(r'SUBREDDIT: r/(\w+)')

# Count the occurrences of each category
category_counts = df['subreddit_category'].value_counts()

# Print the total number of unique categories
total_unique_categories = len(category_counts)
print("Total Unique Categories:", total_unique_categories)

# Print number of entries per category
print("\nCategory Counts:")
print(category_counts)

Total Unique Categories: 29

Category Counts:
subreddit_category
relationships          63324
AskReddit              15440
relationship_advice     8691
tifu                    7685
dating_advice           2849
personalfinance         2312
Advice                  2088
legaladvice             1997
offmychest              1582
loseit                  1452
jobs                    1084
self                    1048
BreakUps                 838
askwomenadvice           688
dogs                     638
running                  567
pettyrevenge             548
needadvice               528
travel                   452
Parenting                435
weddingplanning          433
Pets                     366
Dogtraining              362
cats                     324
AskDocs                  283
college                  264
GetMotivated             169
books                    161
Cooking                  114
Name: count, dtype: int64


Now I want to delete 40% of all subreddits in theor corresponding categories while maintaining the proportions.

In [ ]:
# Count the number of words in each row
df['word_count'] = df['prompt'].str.split().str.len()

# Calculate the median word count
median_word_count = df['word_count'].median()

# Display the results
df[['prompt', 'word_count']].head()

,prompt,word_count
0,SUBREDDIT: r/relationships\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mother has been out of my life since then so there's been alot of progress. Being stronger after learning some lessons there been more insight about that time of my life but when I see him or a picture everything comes back. The emotions and memories bring me back down. \n\nHis friends (both girls) are on my facebook because we get along well which is hard to find and I know they'll always have his back. But seeing him in a picture or talking to him at a convention having a conversation is tough. Crying confront of my current boyfriend is something I want to avoid. \n\nSo I've been thinking that I have to cut contact with these girls because it's time to move on because it's healthier. It's best to avoid him as well. But will they be insulted? Will they accept it? Is there going to be awkwardness? I'm not sure if it's the right to do and could use some outside opinions.\nTL;DR:,356
1,"SUBREDDIT: r/loseit\nTITLE: SV & NSV! Keeping on keeping on.\nPOST: 30F, 5'6"". SW: 236 GW: 150 CW: 219\n\nI weigh myself weekly and measure myself monthly. I'd hit a plateau the last four weeks or so where I was stuck at 222. Felt like kind of a bummer, but knew it's because I haven't been as strict as I should with my diet, and the last week and a half have been crazy with life things, so I haven't been exercising as frequently as I've gotten used to. When I weighed myself as normal on Monday, I was kind of disappointed to see the scale not budging and figured it was time to buckle down again and really watch my diet. Today was my measure-in day, and I've felt cruddy in general since Monday because I caught some chest congestion/cold bug over the weekend. I get on the scale...it says 219. Whaaaaat? I take my measurements, which are down slightly from last month, and with an total-body loss of 8 inches from my starting point on 12/23/14! Some of my clothes have been feeling a bit looser as of late and now I know it's just not in my head. I'm now the lightest and smallest I've been since right around high school!\nTL;DR:",215
2,"SUBREDDIT: r/relationships\nTITLE: Me [19F] with my friend [19M] 10 months, Insecurities - Show or Tell?\nPOST: What are your stories about insecurities you've had in past relationships? How have you dealt with them, particularly the ones that you can't hide?\n\nI'm not currently in a relationship, but recently I've realized that there is someone who likes me, and I'm interested in them, too. Frankly, the only reason I'm not asking them out is because I know that I have some insecurities that need to be worked through - particularly in the realm of body image. While I'm confident in the rest of my body, I've had terrible, awful acne both on my arms and breasts since I was very young. It's a special type with no complete cure, but doctors suggested that I keep my skin oiled until it goes away (dryness irritates it). Because of this it's not so much present anymore as large clusters of scars are.\n\nWould I warn someone about this upfront b

In [ ]:
print(f"Median Word Count: {median_word_count}")

Median Word Count: 266.0


In [ ]:
# Since we want to keep the proportions of long and short posts, we make a new column with the posts word lengths
df['length'] = df['prompt'].str.len()

# Set seed for random selection of calculated remaining posts
SEED = 42
PERCENTAGE = 0.1 # 10% of data

# Empty list where we append the remaining posts later
remaining_posts = []

# Iterate over every category and keep 60% of the posts for each category
for category, group in df.groupby('subreddit_category'):

    # Divide the posts into two groups # TODO average length
    short_posts = group[group['length'] < median_word_count]
    long_posts = group[group['length'] >= median_word_count]

    # Calculate how many of short and longs posts are being kept
    num_short_to_keep = int(PERCENTAGE * (len(short_posts)))
    num_long_to_keep = int(PERCENTAGE * (len(long_posts)))

    # Select the short and longs posts randomly
    selected_short_posts = short_posts.sample(n=num_short_to_keep, random_state=SEED)
    selected_long_posts = long_posts.sample(n=num_long_to_keep, random_state=SEED)

    # Append remaining posts
    remaining_posts.append(pd.concat([selected_short_posts, selected_long_posts]))

# Store the remaining posts in a new df
shortened_df = pd.concat(remaining_posts, ignore_index=True)

print("Number of remaining posts:", len(shortened_df))

Number of remaining posts: 11653


As we can see we significantly reduced the number of rows we are going to continue to work with

In [ ]:
shortened_df[:5]

,prompt,subreddit_category,word_count,length
0,"SUBREDDIT: r/Advice\nTITLE: I'm 23, Moved back in with parents. How do I make new friends reddit?\nPOST: Ok, Ill give you the low down. \n\nI was fairly popular in school always surrounded by friends, but as we all left for university my friendship group separated dramatically. I took 2 gap years and worked as a Teaching assistant. All of my co-workers were older and well.. teachers so I didn't have much of a social life from there.\n\nI then went to Uni again, tonnes of friends until I had to leave due to family shit and stress. My friends are still there. \n\nI haven't been able to go and visit, as I've started a new job (last sept). I work as part of a team of 6 , in estate. My co-workers are nice but not my couple of tea. Plus they have a friendship group established, and In a commission based job, nobody is really your friend anyway.\n\nLastly, My best friend has been with me for 6 years. We made the horrible horrible horrible mistake of hooking up 3 month ago and she broke up with me last night, so she could be single.. Turns out shes back with her ex.... yep. \n\nSo. In a world where I now feel like I have no friends. How do I start to rebuild my circle?\n\nThanks,\nTL;DR:",Advice,228,1185
1,"SUBREDDIT: r/Advice\nTITLE: Always feel an overwhelming lack of confidence while doing things which I excell at and have been doing for years.\nPOST: I don't want to get into specifics, but I've been working in my field for over 6 years. I've accomplished a lot, I've been recognized as good at what I do by superiors and coworkers and I've got a lot of accomplishments and things I've done that should make me more confident and secure, but I still feel nervous, unprepared and anxious before work.\n\nI can't explain it, but I feel anxious and nervous every day before work and during the day even though I should feel comfortable and confident. I love what I do, but I always feel anxious or someone is going to tell me I'm doing a bad job, even though this has never happened.\nTL;DR:",Advice,143,784
2,"SUBREDDIT: r/Advice\nTITLE: Dealing with this break up\nPOST: Hey there,\n\nThis morning I broke up with my girlfriend after 5+ months of having a relationship. This might not be a long time, plus we live in different countries and never once touched each other during this whole relationship, yet I feel broken and she is worse. I talked to her best friend who is really mad at me(understandable of course) and she says that Lucy(my girlfriend) is really in love with me and right now she is a mess; angry, sad, frustrated, betrayed, confused and unloved.\nI get all of that and I feel like the worst human being there is for doing that to her. I still care so much for her and knowing I just kinda destroyed her made me cry all day so far.\nThe reason I broke up with her is that I no longer saw a future for us together. Not without her or me changing, and I also realized I didnt really want to change for her... \nDoes anyone have any idea how I can make this easier for her? What is the best thing I can do right now? I dont need advice for myself, I started this and I feel like I deserve to feel the way I do. But I want to make this easier for her, and I dont know how...\nTL;DR:",Advice,233,1181
3,"SUBREDDIT: r/Advice\nTITLE: Disagreement when picking rooms in the new house\nPOST: My house next year has one basement room no one wants as it's on a floor by itself and next to the kitchen. No one wants to be alone on a floor by themselves with the kitchen and living room and if people are pre drinking, coming into the kitchen you'll be in the room next door. Every room is en-suite except the 2 in the attic which share. No one wants the basement and people don't want but don't mind the attic. It's 2 guys and 8 girls in my house and we agree that 2 girls will take the attic as they work together and hang out a lot. \n \nEventually we decide to draw names and numbers out of a hat for rooms is the most fair 

Now we can try to preprocess this data

In [ ]:
# Function to clean up each entry
def remove_heads_and_tails(text):
    # Find the position of the relevant sections
    start = text.find("TITLE") + len("TITLE ")
    end = text.rfind("TL;DR") if "TL;DR" in text else len(text)

    # Extract the content between TITLE and TL;DR
    cleaned = text[start:end].strip()

    # Remove 'POST:' sections followed by any number of '*' characters or spaces
    cleaned = re.sub(r'POST:\s*\*+\s*', '', cleaned)  # Handles "POST: ****" and variations

    # Remove 'POST:' with no asterisks (with or without spaces)
    cleaned = re.sub(r'POST:\s*', '', cleaned)  # Handles "POST: " with or without spaces

    # Remove all newlines (single and multiple) and replace with a space
    cleaned = re.sub(r'\n+', ' ', cleaned)  # Replace multiple newlines with a single space
    cleaned = cleaned.replace('\n', ' ')  # Replace any remaining single newlines with a space

    # Remove the occurrence of '.\nTL;DR:' where TL;DR comes after a period and newline
    cleaned = re.sub(r'\.\nTL;DR:', '', cleaned)

    # Normalize whitespace (removes extra spaces between words)
    cleaned = ' '.join(cleaned.split())

    return cleaned

# Apply the cleaning function to the content column
shortened_df['shortened'] = shortened_df['prompt'].apply(remove_heads_and_tails)
shortened_df = shortened_df.drop(columns=['prompt', 'length'])

In [ ]:
shortened_df.head()

,subreddit_category,word_count,shortened
0,Advice,228,"I'm 23, Moved back in with parents. How do I make new friends reddit? Ok, Ill give you the low down. I was fairly popular in school always surrounded by friends, but as we all left for university my friendship group separated dramatically. I took 2 gap years and worked as a Teaching assistant. All of my co-workers were older and well.. teachers so I didn't have much of a social life from there. I then went to Uni again, tonnes of friends until I had to leave due to family shit and stress. My friends are still there. I haven't been able to go and visit, as I've started a new job (last sept). I work as part of a team of 6 , in estate. My co-workers are nice but not my couple of tea. Plus they have a friendship group established, and In a commission based job, nobody is really your friend anyway. Lastly, My best friend has been with me for 6 years. We made the horrible horrible horrible mistake of hooking up 3 month ago and she broke up with me last night, so she could be single.. Turns out shes back with her ex.... yep. So. In a world where I now feel like I have no friends. How do I start to rebuild my circle? Thanks,"
1,Advice,143,"Always feel an overwhelming lack of confidence while doing things which I excell at and have been doing for years. I don't want to get into specifics, but I've been working in my field for over 6 years. I've accomplished a lot, I've been recognized as good at what I do by superiors and coworkers and I've got a lot of accomplishments and things I've done that should make me more confident and secure, but I still feel nervous, unprepared and anxious before work. I can't explain it, but I feel anxious and nervous every day before work and during the day even though I should feel comfortable and confident. I love what I do, but I always feel anxious or someone is going to tell me I'm doing a bad job, even though this has never happened."
2,Advice,233,"Dealing with this break up Hey there, This morning I broke up with my girlfriend after 5+ months of having a relationship. This might not be a long time, plus we live in different countries and never once touched each other during this whole relationship, yet I feel broken and she is worse. I talked to her best friend who is really mad at me(understandable of course) and she says that Lucy(my girlfriend) is really in love with me and right now she is a mess; angry, sad, frustrated, betrayed, confused and unloved. I get all of that and I feel like the worst human being there is for doing that to her. I still care so much for her and knowing I just kinda destroyed her made me cry all day so far. The reason I broke up with her is that I no longer saw a future for us together. Not without her or me changing, and I also realized I didnt really want to change for her... Does anyone have any idea how I can make this easier for her? What is the best thing I can do right now? I dont need advice for myself, I started this and I feel like I deserve to feel the way I do. But I want to make this easier for her, and I dont know how..."
3,Advice,398,"Disagreement when picking rooms in the new house My house next year has one basement room no one wants as it's on a floor by itself and next to the kitchen. No one wants to be alone on a floor by themselves with the kitchen and living room and if people are pre drinking, coming into the kitchen you'll be in the room next door. Every room is en-suite except the 2 in the attic which share. No one wants the basement and people don't want but don't mind the attic. It's 2 guys and 8 girls in my house and we agree that 2 girls will take the attic as they work together and hang out a lot. Eventually we decide to draw names and numbers out of a hat for rooms is the most fair method. One of the more vocal girls gets the basement and declares that she refuses to go into that room. She states that ""as a girl"" she doesn't want to be alone on the floor, it's scary for a girl to be 

In [ ]:
# spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "tagger", "textcat"])

def clean_text_spacy(text):
    doc = nlp(text)  # Process the text
    cleaned_tokens = [
        token.text.lower()  # Convert to lowercase
        for token in doc
    ]
    return ' '.join(cleaned_tokens)

tqdm.pandas(desc="Processing texts")

shortened_df['preprocessed'] = shortened_df['shortened'].progress_apply(clean_text_spacy)

preprocessed_df = shortened_df[['subreddit_category', 'word_count', 'preprocessed']]

Processing texts:   0%|          | 0/11653 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
preprocessed_df.head()

,subreddit_category,word_count,preprocessed
0,Advice,228,"i 'm 23 , moved back in with parents . how do i make new friends reddit ? ok , ill give you the low down . i was fairly popular in school always surrounded by friends , but as we all left for university my friendship group separated dramatically . i took 2 gap years and worked as a teaching assistant . all of my co - workers were older and well .. teachers so i did n't have much of a social life from there . i then went to uni again , tonnes of friends until i had to leave due to family shit and stress . my friends are still there . i have n't been able to go and visit , as i 've started a new job ( last sept ) . i work as part of a team of 6 , in estate . my co - workers are nice but not my couple of tea . plus they have a friendship group established , and in a commission based job , nobody is really your friend anyway . lastly , my best friend has been with me for 6 years . we made the horrible horrible horrible mistake of hooking up 3 month ago and she broke up with me last night , so she could be single .. turns out she s back with her ex .... yep . so . in a world where i now feel like i have no friends . how do i start to rebuild my circle ? thanks ,"
1,Advice,143,"always feel an overwhelming lack of confidence while doing things which i excell at and have been doing for years . i do n't want to get into specifics , but i 've been working in my field for over 6 years . i 've accomplished a lot , i 've been recognized as good at what i do by superiors and coworkers and i 've got a lot of accomplishments and things i 've done that should make me more confident and secure , but i still feel nervous , unprepared and anxious before work . i ca n't explain it , but i feel anxious and nervous every day before work and during the day even though i should feel comfortable and confident . i love what i do , but i always feel anxious or someone is going to tell me i 'm doing a bad job , even though this has never happened ."
2,Advice,233,"dealing with this break up hey there , this morning i broke up with my girlfriend after 5 + months of having a relationship . this might not be a long time , plus we live in different countries and never once touched each other during this whole relationship , yet i feel broken and she is worse . i talked to her best friend who is really mad at me(understandable of course ) and she says that lucy(my girlfriend ) is really in love with me and right now she is a mess ; angry , sad , frustrated , betrayed , confused and unloved . i get all of that and i feel like the worst human being there is for doing that to her . i still care so much for her and knowing i just kinda destroyed her made me cry all day so far . the reason i broke up with her is that i no longer saw a future for us together . not without her or me changing , and i also realized i did nt really want to change for her ... does anyone have any idea how i can make this easier for her ? what is the best thing i can do right now ? i do nt need advice for myself , i started this and i feel like i deserve to feel the way i do . but i want to make this easier for her , and i do nt know how ..."
3,Advice,398,"disagreement when picking rooms in the new house my house next year has one basement room no one wants as it 's on a floor by itself and next to the kitchen . no one wants to be alone on a floor by themselves with the kitchen and living room and if people are pre drinking , coming into the kitchen you 'll be in the room next door . every room is en - suite except the 2 in the attic which share . no one wants the basement and people do n't want but do n't mind the attic . it 's 2 guys and 8 girls in my house and we agree that 2 girls will take the attic as they work together and hang out a lot . eventually we decide to draw names and numbers out of a hat for rooms is the most fair method . one of the more vocal girls gets the basement and declares that she refuses to go into that r

In [ ]:
file_path = f'..\\resources\\preprocessed_df_{PERCENTAGE:.0%}_data.tsv'

try:
    preprocessed_df.to_csv(file_path, index=False, sep="\t")
    print(f"File saved successfully: {file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

File saved successfully: ..\resources\preprocessed_df_10%_data.tsv


In [ ]:
#download preprocessed
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

okay! now we can run it over the roberta_toxicity_classifier!

In [ ]:
try:
    processed_df = pd.read_csv(file_path, sep="\t")
except Exception as e:
    print(f"Error loading file: {e}")

In [ ]:
processed_df[:2]

,subreddit_category,word_count,preprocessed
0,Advice,228,"i 'm 23 , moved back in with parents . how do i make new friends reddit ? ok , ill give you the low down . i was fairly popular in school always surrounded by friends , but as we all left for university my friendship group separated dramatically . i took 2 gap years and worked as a teaching assistant . all of my co - workers were older and well .. teachers so i did n't have much of a social life from there . i then went to uni again , tonnes of friends until i had to leave due to family shit and stress . my friends are still there . i have n't been able to go and visit , as i 've started a new job ( last sept ) . i work as part of a team of 6 , in estate . my co - workers are nice but not my couple of tea . plus they have a friendship group established , and in a commission based job , nobody is really your friend anyway . lastly , my best friend has been with me for 6 years . we made the horrible horrible horrible mistake of hooking up 3 month ago and she broke up with me last night , so she could be single .. turns out she s back with her ex .... yep . so . in a world where i now feel like i have no friends . how do i start to rebuild my circle ? thanks ,"
1,Advice,143,"always feel an overwhelming lack of confidence while doing things which i excell at and have been doing for years . i do n't want to get into specifics , but i 've been working in my field for over 6 years . i 've accomplished a lot , i 've been recognized as good at what i do by superiors and coworkers and i 've got a lot of accomplishments and things i 've done that should make me more confident and secure , but i still feel nervous , unprepared and anxious before work . i ca n't explain it , but i feel anxious and nervous every day before work and during the day even though i should feel comfortable and confident . i love what i do , but i always feel anxious or someone is going to tell me i 'm doing a bad job , even though this has never happened ."


In [ ]:
#Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('s-nlp/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('s-nlp/roberta_toxicity_classifier')

#Set the model to evaluation mode
model.eval()

#Function to classify toxicity
def classify_toxicity(text):
    # Tokenize the input text
    batch = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=512)

    # Move inputs to GPU if available
    if torch.cuda.is_available():
        batch = batch.to('cuda')
        model.to('cuda')  # Move model to GPU

    #Get the model output
    with torch.no_grad():
        output = model(batch)

# Apply sigmoid to logits and get the predicted label
    predicted_label = torch.sigmoid(output.logits).argmax().item()

    return predicted_label

# Use tqdm to display a progress bar
tqdm.pandas(desc="Processing toxicity labels")

#Apply the toxicity classifier to each cleaned content post with a progress bar
processed_df['toxicity_label'] = processed_df['preprocessed'].progress_apply(classify_toxicity)

#Keep 'subreddit_category', 'word_count', 'cleaned_content', and 'toxicity_label' columns
final_df = processed_df[['subreddit_category', 'word_count', 'preprocessed', 'toxicity_label']]

processed_data_path = f'..\\resources\\toxicity_analysis_{PERCENTAGE:.0%}_data.tsv'

try:
    final_df.to_csv(processed_data_path, index=False, sep="\t")
    print(f"File saved successfully: {file_path}")
except Exception as e:
    print(f"Error saving file: {e}")

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processing toxicity labels:   0%|          | 0/11653 [00:00<?, ?it/s]

File saved successfully: ..\resources\preprocessed_df_10%_data.tsv


In [ ]:
final_df['toxicity_label'].unique()
value_counts = final_df['toxicity_label'].value_counts()
print(value_counts)

toxicity_label
0    11165
1      488
Name: count, dtype: int64


In [ ]:
final_df.head()

,subreddit_category,word_count,preprocessed,toxicity_label
0,Advice,228,"i 'm 23 , moved back in with parents . how do i make new friends reddit ? ok , ill give you the low down . i was fairly popular in school always surrounded by friends , but as we all left for university my friendship group separated dramatically . i took 2 gap years and worked as a teaching assistant . all of my co - workers were older and well .. teachers so i did n't have much of a social life from there . i then went to uni again , tonnes of friends until i had to leave due to family shit and stress . my friends are still there . i have n't been able to go and visit , as i 've started a new job ( last sept ) . i work as part of a team of 6 , in estate . my co - workers are nice but not my couple of tea . plus they have a friendship group established , and in a commission based job , nobody is really your friend anyway . lastly , my best friend has been with me for 6 years . we made the horrible horrible horrible mistake of hooking up 3 month ago and she broke up with me last night , so she could be single .. turns out she s back with her ex .... yep . so . in a world where i now feel like i have no friends . how do i start to rebuild my circle ? thanks ,",0
1,Advice,143,"always feel an overwhelming lack of confidence while doing things which i excell at and have been doing for years . i do n't want to get into specifics , but i 've been working in my field for over 6 years . i 've accomplished a lot , i 've been recognized as good at what i do by superiors and coworkers and i 've got a lot of accomplishments and things i 've done that should make me more confident and secure , but i still feel nervous , unprepared and anxious before work . i ca n't explain it , but i feel anxious and nervous every day before work and during the day even though i should feel comfortable and confident . i love what i do , but i always feel anxious or someone is going to tell me i 'm doing a bad job , even though this has never happened .",0
2,Advice,233,"dealing with this break up hey there , this morning i broke up with my girlfriend after 5 + months of having a relationship . this might not be a long time , plus we live in different countries and never once touched each other during this whole relationship , yet i feel broken and she is worse . i talked to her best friend who is really mad at me(understandable of course ) and she says that lucy(my girlfriend ) is really in love with me and right now she is a mess ; angry , sad , frustrated , betrayed , confused and unloved . i get all of that and i feel like the worst human being there is for doing that to her . i still care so much for her and knowing i just kinda destroyed her made me cry all day so far . the reason i broke up with her is that i no longer saw a future for us together . not without her or me changing , and i also realized i did nt really want to change for her ... does anyone have any idea how i can make this easier for her ? what is the best thing i can do right now ? i do nt need advice for myself , i started this and i feel like i deserve to feel the way i do . but i want to make this easier for her , and i do nt know how ...",0
3,Advice,398,"disagreement when picking rooms in the new house my house next year has one basement room no one wants as it 's on a floor by itself and next to the kitchen . no one wants to be alone on a floor by themselves with the kitchen and living room and if people are pre drinking , coming into the kitchen you 'll be in the room next door . every room is en - suite except the 2 in the attic which share . no one wants the basement and people do n't want but do n't mind the attic . it 's 2 guys and 8 girls in my house and we agree that 2 girls will take the attic as they work together and hang out a lot . eventually we decide to draw names and numbers out of a hat for rooms is the most fair method . one of the more vocal girls gets the basement and declares that she refu

In [ ]:
# download
from google.colab import files
files.download(processed_data_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>